# Generic Engine Pipeline

The notebook provides a generic end-to-end model training pipeline for experimentation.

The notebook works by configuring the input data set and uses a disk caching method to cache function calls to speed up model training.

Unlike prior work on Project Echo, no offline pre-processing of the audio files is required (this is encapsulted in the processing pipeline within this notebook).

Author: akudilczak

In [ ]:
########################################################################################
# library imports
########################################################################################

# disable warnings
import warnings
warnings.filterwarnings("ignore")

# generic libraries
from platform import python_version
import functools
from functools import lru_cache
import diskcache as dc
import hashlib
import numpy as np
import os
import datetime
import random
import matplotlib.pyplot as plt

# tensor flow / keras related libraries
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_hub as hub
import tensorflow_addons as tfa 
from keras.utils import dataset_utils

# image processing related libraries
import librosa

# audio processing libraries
import audiomentations
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# print system information
print('Python Version           : ', python_version())
print('TensorFlow Version       : ', tf.__version__)
print('TensorFlow IO Version    : ', tfio.__version__)
print('Librosa Version          : ', librosa.__version__)
print('Audiomentations Version  : ', audiomentations.__version__)

## System Configuration

The following code sets up the pipeline with configuration options.

The key is to set the audio data directory to the root directory containing the folders with raw audio files. 

This expects the folders names to be the species names.  

In [ ]:
########################################################################################
# system constants
########################################################################################

SC = {
    'AUDIO_DATA_DIRECTORY': "d:\\data\\bc",
    'CACHE_DIRETORY': "d:\\pipeline_cache",

    'AUDIO_CLIP_DURATION': 5, # 5 second clips
    'AUDIO_NFFT': 2000,
    'AUDIO_WINDOW': 500,
    'AUDIO_STRIDE': 200,
    'AUDIO_SAMPLE_RATE': 32000,  # int(44100/2)
    'AUDIO_MELS': 260,
    'AUDIO_FMIN': 10,
    'AUDIO_FMAX': int(32000 / 2),
    'AUDIO_TOP_DB': 80,

    'MODEL_INPUT_IMAGE_WIDTH': 260,
    'MODEL_INPUT_IMAGE_HEIGHT': 260,
    'MODEL_INPUT_IMAGE_CHANNELS': 3,

    'USE_DISK_CACHE': True,
    'SAMPLE_VARIANTS': 40,
    'CLASSIFIER_BATCH_SIZE': 8
}

In [ ]:
def enforce_memory_limit(mem_mb):
  # enforce memory limit on GPU

  gpus = tf.config.experimental.list_physical_devices('GPU')
  if gpus:
    try:
      tf.config.experimental.set_virtual_device_configuration(
          gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=mem_mb)])
      print(f"vram limit set to {mem_mb}MB")
    except RuntimeError as e:
      print(e)
      
# enforce max 5GB memory on GPU for this notebook if you have a small GPU
# enforce_memory_limit(5120)

# Disk Caching

The following code creates a disk cache.  You will need lots of space (20 GB+) if you create large melspectrograms.

The caching works by serialising a function call signature and hashing it into a key.  This key is used to store the result of the function call.

This allows the a result from the cache to be utilised instead of calling the function, which means the entire data processing pipeline can be cached if used correctly.

This works best when the function being cached is idempotent.  There may be circumstances where it doesn't matter.  Be careful with using this cache as you may get unexpected results.

In [ ]:
########################################################################################
# Create a DiskCache instance
# This cache will allow us store intermediate function results to speed up the 
# data processing pipeline
########################################################################################
cache = dc.Cache(SC['CACHE_DIRETORY'], cull_limit=0, size_limit=10**9) 


########################################################################################
# a helper function to create a hash key from a function signature and arguments
########################################################################################
def create_function_key(func, *args, **kwargs):
    partial_func = functools.partial(func, *args, **kwargs)
    func_name = partial_func.func.__name__
    func_module = partial_func.func.__module__
    args_repr = repr(partial_func.args)
    kwargs_repr = repr(sorted(partial_func.keywords.items()))

    key = f"{func_module}.{func_name}:{args_repr}:{kwargs_repr}"
    # Use hashlib to create a hash of the key for shorter and consistent length
    key_hash = hashlib.sha256(key.encode()).hexdigest()

    return key, key_hash, partial_func


########################################################################################
# Execute a function and cache the result
# If already executed, retrieve function output from the cache instead
########################################################################################
def execute_cached_function(func, *args, **kwargs):
    key_string,key,partial_func = create_function_key(func, *args, **kwargs)
    # Check if the result is in the cache
    if key in cache:
        result = cache[key]
        # print(f"Result loaded from cache key: {key}")
    else:
        # If not in cache, call the slow operation and store the result in cache
        result = partial_func()
        cache[key] = result
        #print(f"New result calculated and stored in cache key: {key}")
    return result

# Loading the files into a Tensorflow dataset structure for model training

This initial code loads only the filenames.  The filenames are then split into train, validation and test datasets.  This is designed deliberately this way to conserve runtime memory.

Subsequent downstream loading of the file content occurs as part of the data pipeline transformation 'map' function.  See below.

In [ ]:
########################################################################################
# these helper functions load the audio data into a 'dataset' using only paths
# just dealing with paths at this early stage means the entire dataset can be shuffled in
# memory and split before loading the actual audio data into memory
########################################################################################
def paths_and_labels_to_dataset(image_paths, labels, num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    label_ds = dataset_utils.labels_to_dataset(
        labels, 
        'categorical', 
        num_classes)
    zipped_path_ds = tf.data.Dataset.zip((path_ds, label_ds))
    return zipped_path_ds

def create_datasets(audio_files, train_split=0.7, val_split=0.2):
    file_paths, labels, class_names = dataset_utils.index_directory(
            audio_files,
            labels="inferred",
            formats=('.ogg','.mp3','.wav','.flac'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=False)

    dataset = paths_and_labels_to_dataset(
        image_paths=file_paths,
        labels=labels,
        num_classes=len(class_names))
    
    # Calculate the size of the dataset
    dataset_size = len(dataset)
    
    # Calculate the number of elements for each dataset split
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    test_size = dataset_size - train_size - val_size

    # Shuffle the dataset
    dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)

    # Split the dataset
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size + val_size).take(test_size)
    
    return train_ds, val_ds, test_ds, class_names

In [ ]:
# create the dataset
train_ds, val_ds, test_ds, class_names = create_datasets(SC['AUDIO_DATA_DIRECTORY'],train_split=0.8, val_split=0.19)
print("Class names: ", class_names)
print(f"Training   dataset length: {len(train_ds)}")
print(f"Validation dataset length: {len(val_ds)}")
print(f"Test       dataset length: {len(test_ds)}")

In [ ]:
# show what the pipeline looks like at this stage
for item in train_ds.take(10):
    print(item)

In [ ]:
def load_random_subsection(path, duration_secs):

    # read the file data
    file_contents=tf.io.read_file(path)

    # attempt to decode each known format
    try:
        tmp_audio_t = tfio.audio.decode_flac(input=file_contents)
    except:
        pass

    try:
        tmp_audio_t = tfio.audio.decode_vorbis(input=file_contents)
    except:
        pass
    
    try:
        tmp_audio_t = tfio.audio.decode_mp3(input=file_contents)
    except:
        pass
    
    try:
        tmp_audio_t = tfio.audio.decode_wav(input=file_contents)
    except:
        pass

    # cast and keep left channel only
    tmp_audio_t = tf.cast(tmp_audio_t, tf.float32)[:,-1]
    
    # resample the sample rate
    tmp_audio_t = tfio.audio.resample(tmp_audio_t, tfio.audio.AudioIOTensor(path)._rate.numpy(), SC['AUDIO_SAMPLE_RATE'])

    # Determine the audio file's duration in seconds
    audio_duration_secs = tf.shape(tmp_audio_t)[0] / SC['AUDIO_SAMPLE_RATE']
    
    if audio_duration_secs>duration_secs:
    
        # Calculate the starting point of the 5-second subsection
        max_start = tf.cast(audio_duration_secs - duration_secs, tf.float32)
        start_time_secs = tf.random.uniform((), 0.0, max_start, dtype=tf.float32)
        
        start_index = tf.cast(start_time_secs * SC['AUDIO_SAMPLE_RATE'], dtype=tf.int32)
  
        # Load the 5-second subsection
        end_index = tf.cast(start_index + tf.cast(duration_secs, tf.int32) * SC['AUDIO_SAMPLE_RATE'], tf.int32)
        
        subsection = tmp_audio_t[start_index : end_index]
    
    else:
        # Pad the subsection with silence if it's shorter than 5 seconds
        padding_length = duration_secs * SC['AUDIO_SAMPLE_RATE'] - tf.shape(tmp_audio_t)[0]
        padding = tf.zeros([padding_length], dtype=tmp_audio_t.dtype)
        subsection = tf.concat([tmp_audio_t, padding], axis=0)

    return subsection

In [ ]:
# audio augmentation pipeline
# see https://github.com/iver56/audiomentations for more options
audio_augmentations = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.2),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.2),
])

In [ ]:
# image augmentation pipeline
def image_augmentations(image, label):

    degrees = tf.random.uniform(shape=(1,), minval=-2, maxval=2)
    radians = degrees * 0.017453292519943295  # convert the angle in degree to radians

    image = tfa.image.rotate(image, radians, interpolation='bilinear')
    
    return image,label

In [ ]:
def dataset_melspectro_pipeline(path, label, variant, config, audio_clip):
    
    # Convert to spectrogram
    image = tfio.audio.spectrogram(
        audio_clip,
        nfft=config['AUDIO_NFFT'], 
        window=config['AUDIO_WINDOW'], 
        stride=config['AUDIO_STRIDE'])

    # Convert to melspectrogram
    image = tfio.audio.melscale(
        image, 
        rate=config['AUDIO_SAMPLE_RATE'], 
        mels=config['AUDIO_MELS'], 
        fmin=config['AUDIO_FMIN'], 
        fmax=config['AUDIO_FMAX'])
    
    # reshape into standard 3 channels to add the color channel
    image = tf.expand_dims(image, -1)
    
    # most pre-trained model classifer model expects 3 color channels
    image = tf.repeat(image, config['MODEL_INPUT_IMAGE_CHANNELS'], axis=2)
    
    # calculate the image shape and ensure it is correct
    expected_clip_samples = int(config['AUDIO_CLIP_DURATION'] * config['AUDIO_SAMPLE_RATE'] / config['AUDIO_STRIDE'])
    image = tf.ensure_shape(image, [expected_clip_samples, config['AUDIO_MELS'], config['MODEL_INPUT_IMAGE_CHANNELS']])
    
    # note here a high quality LANCZOS5 is applied to resize the image to match model image input size
    image = tf.image.resize(image, (config['MODEL_INPUT_IMAGE_WIDTH'],config['MODEL_INPUT_IMAGE_HEIGHT']), 
                            method=tf.image.ResizeMethod.LANCZOS5)

    # rescale to range [0,1]
    image = image - tf.reduce_min(image) 
    image = image / (tf.reduce_max(image)+0.0000001)
    
    return image, label

In [ ]:
def dataset_validation_pipeline(path, label, variant, config):
    # this loads a random 5 second clip of audio from the raw audio file
    audio_clip = load_random_subsection(path, duration_secs=config['AUDIO_CLIP_DURATION'])

    # Convert to spectrogram
    image, label = dataset_melspectro_pipeline(path, label, variant, config, audio_clip)
    
    return image, label

def dataset_test_pipeline(path, label, variant, config):
    # this loads a random 5 second clip of audio from the raw audio file
    audio_clip = load_random_subsection(path, duration_secs=config['AUDIO_CLIP_DURATION'])

    # Convert to spectrogram
    image, label = dataset_melspectro_pipeline(path, label, variant, config, audio_clip)
    
    return image, label

def dataset_training_pipeline(path, label, variant, config):
    # this loads a random 5 second clip of audio from the raw audio file
    # note: it also converts the sample rate to AUDIO_SAMPLE_RATE and selects the left channel only
    audio_clip = load_random_subsection(path, duration_secs=config['AUDIO_CLIP_DURATION'])
   
    # apply audio augmentation to the clip
    # note: this augmentation is NOT applied in the test and validation pipelines
    audio_clip = audio_augmentations(samples=audio_clip.numpy(), sample_rate=config['AUDIO_SAMPLE_RATE'])
   
    # Convert to spectrogram
    image, label = dataset_melspectro_pipeline(path, label, variant, config, audio_clip)
    
    # apply image based augmentation to the melspectrogram
    image, label = image_augmentations(image, label)
    
    return image, label

In [ ]:
def dataset_cached_pipeline(pipeline_fn, path, label):
    variant = random.randrange(0, SC['SAMPLE_VARIANTS'])
    if SC['USE_DISK_CACHE']:
        return execute_cached_function(pipeline_fn,path,label,variant,SC)
    else:
        return pipeline_fn(path,label,variant,SC)
    
# this will allow python execution within the tensorflow pipeline
def dataset_pipeline_wrapper(pipeline_fn, path, label):
  
    # Use a lambda function to pass two arguments to the dataset_example_pipeline function
    image, label = tf.py_function(func=lambda x, y: dataset_cached_pipeline(pipeline_fn, x, y), inp=(path, label), Tout=(tf.float32, label.dtype))

    # Set the shape of the output tensors manually
    image.set_shape([SC['MODEL_INPUT_IMAGE_WIDTH'], SC['MODEL_INPUT_IMAGE_HEIGHT'], SC['MODEL_INPUT_IMAGE_CHANNELS']])
    label.set_shape([len(class_names),])  # Set the shape of the label tensor

    return image, label

In [ ]:
########################################################################################
# create the datasets useful for training a classification model
# note: it seems that using py_function prevents the use of parallel calls which slows down
# things siginificantly.  This requires investigation of alternative ways to construct the pipeline
########################################################################################

len_train_ds = len(train_ds)

train_dataset = (train_ds
                 .shuffle(len_train_ds)
                 .map(functools.partial(dataset_pipeline_wrapper, dataset_training_pipeline), num_parallel_calls=1)
                 .batch(SC['CLASSIFIER_BATCH_SIZE'])
                 .repeat(count=10)         
)

validation_dataset = (val_ds
                      .map(functools.partial(dataset_pipeline_wrapper, dataset_validation_pipeline), num_parallel_calls=1)
                      .batch(SC['CLASSIFIER_BATCH_SIZE'])
                      .repeat(count=4)
)

test_dataset = (test_ds
                .map(functools.partial(dataset_pipeline_wrapper, dataset_test_pipeline), num_parallel_calls=1)
                .batch(SC['CLASSIFIER_BATCH_SIZE'])
                .repeat(count=5)  
)

In [ ]:
# show what the pipeline looks like at this stage
for melspectrogram,label in train_dataset.take(1):
    print(f' sample info: {melspectrogram.shape}, {label}')
    for example in range(melspectrogram.shape[0]):
        plt.imshow(melspectrogram[example,:,:,0].numpy().T, cmap='afmhot', origin='lower')
        plt.show()

In [ ]:
def build_model(trainable):
    # build a really simple classification model using a pre-training Efficientnet V2
    model = tf.keras.Sequential(
        [
            # input layer
            tf.keras.layers.InputLayer(input_shape=(SC['MODEL_INPUT_IMAGE_HEIGHT'], 
                                                    SC['MODEL_INPUT_IMAGE_WIDTH'], 
                                                    SC['MODEL_INPUT_IMAGE_CHANNELS'])),
  
            # use the model as a feature generator only
            # needs 384x384x3 images
            # hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2", trainable), 
            
            # needs 260x260x3 images
            hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2", trainable), 
                    
            # add the classification layer here       
            tf.keras.layers.Flatten(), 
            tf.keras.layers.BatchNormalization(),
            
            tf.keras.layers.Dense(len(class_names)*8, 
                                  activation="relu", 
                                  #kernel_regularizer=tf.keras.regularizers.L1(1e-2)
                                  ),
            tf.keras.layers.BatchNormalization(),
            
            tf.keras.layers.Dense(len(class_names)*4, 
                                  activation="relu", 
                                  #kernel_regularizer=tf.keras.regularizers.L1(1e-2)
                                  ),
            tf.keras.layers.BatchNormalization(),
            
            tf.keras.layers.Dropout(0.50),
            tf.keras.layers.Dense(len(class_names), activation=None),
        ]
    )
    # need to tell the model what the input shape is
    model.build([None, 
                 SC['MODEL_INPUT_IMAGE_HEIGHT'],
                 SC['MODEL_INPUT_IMAGE_WIDTH'], 
                 SC['MODEL_INPUT_IMAGE_CHANNELS']])

    # show the model
    model.summary()
    
    return model

In [ ]:
if not os.path.exists('models/'):
    os.mkdir('models/')
    
# allow all the weights to be trained
model = build_model(True)

# the form_logits means the loss function has the 'softmax' buillt in.  This approach is numerically more stable
# than including the softmax activation on the last layer of the classifier
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              metrics=["accuracy"],
              )

# tensorboard for visualisation of results
log_dir = "tensorboard_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      histogram_freq=1)

# reduce learning rate to avoid overshooting local minima
lr_reduce_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                      factor=0.75,
                                                      patience=12, 
                                                      verbose=1,
                                                      mode='min',
                                                      cooldown=0, 
                                                      min_lr=1e-7)

# end the training if no improvement for 16 epochs in a row, then restore best model weights
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=16,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True,
)

# save the best model as it trains..
mcp_save = tf.keras.callbacks.ModelCheckpoint('models/checkpoint_generic_model.hdf5', 
                                           save_best_only=True, 
                                           monitor='val_loss', 
                                           mode='min')

# any changes to the source code will generally require the disk cache to be cleared.
# So to be safe, the cache is cleared before training the model.  If you are sure
# the cache is still valid then comment out this code
# the first few epochs of the model training will be slow as the cache is populated with pipeline samples
# and will depend on the dataset size and the number of variants included
# cache.clear()

# fit the model to the training set
# this may take 12-24 hours to run to full model convergence depending on your machine
model.fit(train_dataset, 
          validation_data=validation_dataset,
          callbacks=[lr_reduce_plateau, early_stopping, tensorboard_callback, mcp_save],
          epochs=10000)      

In [ ]:
# this runs the pipeline and prediction model and returns target and probability
def predict_class(predictions):
   target_index = int(tf.argmax(tf.squeeze(predictions)).numpy())
   target_class = class_names[target_index]    
   target_proba = 100.0*tf.nn.softmax(predictions)[target_index].numpy()
   target_proba = str(round(target_proba, 2))
   return target_class, target_proba

# run prediction on the test entries
print(f'class_names:  {class_names}')
for feature,label in test_dataset:
   
    predictions = model.predict(feature, verbose=0)
    # print(f'predictions : {predictions}')
    
    for b in range(predictions.shape[0]):
        
        true_index = int(tf.argmax(tf.squeeze(label[b])).numpy())
        true_class = class_names[true_index] 
        
        pred_class, pred_proba = predict_class(predictions[b])
        
        print(f'')
        print(f'true_class  : {true_class}')    
        print(f'pred_class  : {pred_class}')
        print(f'pred_proba  : {pred_proba}')

In [ ]:
# save the model weights
model.save_weights('models/generic_engine_pipeline_model.hdf5', save_format='h5')

# test load the model for inference
test_model = build_model(False)
test_model.build([None, 
                SC['MODEL_INPUT_IMAGE_HEIGHT'],
                SC['MODEL_INPUT_IMAGE_WIDTH'], 
                SC['MODEL_INPUT_IMAGE_CHANNELS']])
test_model.load_weights('models/generic_engine_pipeline_model.hdf5')